In [1]:
from keras.models import Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np
from time import time
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Restricting GPU memory
os.environ['CUDA_VISIBLE_DEVICES']='0'
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
import dataloader, TNN, evaluation # Loading helper functions

In [4]:
dev2train, test2train = dataloader.creating_id_mapping()

# Getting i-vectors
trn_bl_ivector, trn_bg_ivector, dev_bl_ivector, dev_bg_ivector, tst_ivector = dataloader.get_ivectors()

# Loading labels for task 2
trn_bl_id, trn_bg_id, dev_bl_id, dev_bg_id, tst_id = dataloader.get_spk_ids()
dev_trials_label = np.append(dev_bl_id,dev_bg_id)

# Making labels for task 1
trn_ivector = np.append(trn_bl_ivector, trn_bg_ivector,axis=0) # combining bg and bl speakers into a single vector
trn_trials = np.append( np.ones([len(trn_bl_ivector), 1]), np.zeros([len(trn_bg_ivector), 1]))
dev_ivector = np.append(dev_bl_ivector, dev_bg_ivector,axis=0) # combining bg and bl
dev_trials = np.append( np.ones([len(dev_bl_id), 1]), np.zeros([len(dev_bg_id), 1]))
tst_trials, tst_trials_label = dataloader.get_tst_trials()

# Models

## PLA model

In [5]:
model1 = LinearDiscriminantAnalysis()
model1.fit(np.append(trn_ivector,dev_ivector, axis=0),np.append(trn_trials,dev_trials))

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [6]:
dev_bl_id_along_trnset = []
for iter in range(len(dev_bl_id)):
    dev_bl_id_along_trnset.extend([dev2train[dev_bl_id[iter]]])
model2 = LinearDiscriminantAnalysis()
model2.fit(np.append(trn_bl_ivector,dev_bl_ivector, axis=0),np.append(trn_bl_id,dev_bl_id_along_trnset))

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

# Prediction phase

## Evaluating on testset

In [7]:
start = time()
task1_pred = model1.predict_proba(tst_ivector)
err = dataloader.calculate_EER(tst_trials,task1_pred[:,1])
print(f'time used = {time()-start:.3}')

Top S detector EER is 10.67%
time used = 0.0184


In [8]:
start = time()
task2_pred = model2.predict(tst_ivector)
tst_trials_confusion = dataloader.get_trials_label_with_confusion(task2_pred, tst_trials_label, test2train, tst_trials )
dev_EER_confusion = dataloader.calculate_EER_with_confusion(task1_pred[:,1],tst_trials_confusion)
print(f'time used = {time()-start:.3}')

Top 1 detector EER is 12.37% (Total confusion error is 230)
time used = 0.587
